In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import math
from definitions import *
import cv2

In [14]:
masks_df = []
expressions = os.listdir(PROCESSED_PATH)
for exp in expressions:
    if(len(exp.split("."))>1):
        continue
    types = os.listdir(PROCESSED_PATH + "/" + exp)
    for tp in types:
        if(len(tp.split("."))>1):
            continue
        df = pd.read_csv(PROCESSED_PATH + "/" + exp+"/"+tp+"/mean-lms-"+exp+"-"+tp+".csv",index_col=0)
        masks_df.append([exp+"-"+tp,df])

In [20]:
dists_df = pd.read_csv("./masks_average_dists.csv",index_col=0)

In [15]:
masks_df

[['00-00',
          x     y
  0    1249  1057
  1    1247   845
  2    1248   913
  3    1199   649
  4    1246   783
  ..    ...   ...
  463  1391   510
  464  1358   523
  465  1341   531
  466  1652   487
  467  1680   470
  
  [468 rows x 2 columns]],
 ['00-01',
          x     y
  0    1244  1060
  1    1241   849
  2    1244   916
  3    1196   652
  4    1241   787
  ..    ...   ...
  463  1390   513
  464  1358   526
  465  1340   533
  466  1653   490
  467  1681   472
  
  [468 rows x 2 columns]],
 ['01-00',
          x     y
  0    1242  1071
  1    1239   867
  2    1242   927
  3    1194   665
  4    1239   805
  ..    ...   ...
  463  1390   511
  464  1357   526
  465  1340   536
  466  1652   485
  467  1680   470
  
  [468 rows x 2 columns]],
 ['01-01',
          x     y
  0    1246  1067
  1    1243   864
  2    1245   924
  3    1196   662
  4    1242   802
  ..    ...   ...
  463  1392   510
  464  1359   524
  465  1341   534
  466  1653   485
  467  1681   471
  

# Heatmap nos pontos

In [35]:
for exp_tp, mask_df in tqdm(masks_df):
    if(exp_tp == "00-00"):
        continue
    exp,tp = exp_tp.split("-")
    image = np.zeros(
            (int(final_image_size_height), int(final_image_size_width), 3), dtype=float)
    x = (mask_df["x"]).tolist()
    y = (mask_df["y"]).tolist()
    lms = []
    for i in range(len(x)):
        lms.append([int(x[i]), int(y[i])])
    for i,lm in enumerate(lms):
        max_esc = dists_df[exp_tp].max() - dists_df[exp_tp].min()
        # vermelho
        #   min   -   0
        #   max   -   255
        #   val   -   x
        #       x = val*255/max
        # azul
        #   min   -   255
        #   max   -   0
        #   val   -   x
        #       x = 255 - val*255/max
        red = float(dists_df[exp_tp].tolist()[i])*255/float(max_esc)
        blue = 255 - red
        image = cv2.circle(image, (lm), radius=5, color=(int(blue),0, int(red)), thickness=-1)
    '''
    for p in points_df["points_above_average"].tolist():
        image = cv2.circle(image, (lms[p]), radius=5, color=(0, 0, 255), thickness=-1)
        #cv2.putText(image, '.', (lms[p]), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)  
    '''    
    cv2.imwrite("./heatmap_dists_by_points/"+exp_tp+".png", image)

100%|██████████| 16/16 [00:02<00:00,  5.80it/s]


# Heatmap nos triangulos